# Import

In [10]:
pip install rdflib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
import rdflib
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD
from rdflib import Namespace
from rdflib import URIRef
from SPARQLWrapper import SPARQLWrapper, JSON

#!pip install pandas
import pandas as pd

#!pip install numpy
import numpy as np

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Functions

In [12]:
# endpoint
icon_sparql = "https://projects.dharc.unibo.it/icondataset/sparql"

In [13]:
def sparql_query_setting(query, endpoint):# set the endpoint 
  sparql = SPARQLWrapper(endpoint)                 
  # set the query
  sparql.setQuery(query)
  # set the returned format
  sparql.setReturnFormat(JSON)
  # get the results
  results = sparql.query().convert()
  return results

# Research Questions

## Cultural Phenomena

### Q1. 
How many artworks have a cultural phenomenon associated?

In [14]:
q_art_cf = """
PREFIX d: <https://w3id.org/icon/data/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
PREFIX sim: <https://w3id.org/simulation/ontology/>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT (count(distinct ?artwork) as ?tot) WHERE {
    
    ?iconol icon:aboutWorkOfArt ?artwork;  
        icon:recognizedIntrinsicMeaning / icon:recognizedCulturalPhenomenon ?lev3. 

} 
""" 

result_art_cf = sparql_query_setting(q_art_cf, icon_sparql)
for b in result_art_cf['results']['bindings']:
    print(b["tot"]["value"])

324


### Q2. 
What is the frequency of level 1 and 2 subjects occurring with each cultural phenomenon? 

In [22]:
# frequency of level 1 subjects

cf1_list = []
q_cf1 = """
PREFIX d: <https://w3id.org/icon/data/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
PREFIX sim: <https://w3id.org/simulation/ontology/>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>

SELECT DISTINCT ?artwork ?lev1Label ?CFLabel  WHERE {
    VALUES ?rel {icon:hasCharacter icon:hasEvent icon:hasNamedObject icon:hasPlace icon:hasPersonification icon:hasSymbol}
    ?meaning ^icon:recognizedCulturalPhenomenon / ^icon:recognizedIntrinsicMeaning ?iconol .
    ?meaning a icon:CulturalPhenomenon; rdfs:label ?CFLabel. 
    ?iconol icon:aboutWorkOfArt ?artwork .
    ?rec icon:aboutWorkOfArt ?artwork.
    ?rec a icon:PreiconographicalRecognition. 
    {?rec icon:recognizedArtisticMotif ?am} UNION {?rec icon:recognizedComposition ?comp. ?comp icon:hasPart ?am}
    ?am icon:hasFactualMeaning | icon:hasExpressionalMeaning | crm:P138_represents ?lev1. 
    ?lev1 rdfs:label ?lev1Label.

} 
"""

result_cf1 = sparql_query_setting(q_cf1, icon_sparql)
for b in result_cf1['results']['bindings']:
    art = (b["artwork"]["value"])
    subj = (b["lev1Label"]["value"])
    cf = (b["CFLabel"]["value"])
    if cf != '?':
        cf1_list.append([art, subj, cf])

cf1 = pd.DataFrame(np.array(cf1_list),
    columns=['Artwork', 'Level 1 subject', 'Cultural Phenomenon'])

cf1_count = cf1.groupby(["Cultural Phenomenon", "Level 1 subject"])["Artwork"].count().reset_index().sort_values(by="Artwork", ascending=False)

cf1_count.head(15)

,Cultural Phenomenon,Level 1 subject,Artwork
3564,Representation of classical content with conte...,man,15
1940,Iconographic evolution of the representation o...,head of a lion,11
1939,Iconographic evolution of the representation o...,head of a dog,11
1941,Iconographic evolution of the representation o...,head of a wolf,11
3611,Representation of classical content with conte...,woman,9
3297,Reintegration of classical style and content i...,man,9
1852,Iconographic evolution of classical subjects,man,9
1783,Iconographic evolution of blind Cupid: it deve...,bow,8
1781,Iconographic evolution of blind Cupid: it deve...,arrow,8
1848,Iconographic evolution of classical subjects,head of a wolf,8


In [21]:
# frequency of level 2 subjects
cf2_list = []
q_cf2 = """
PREFIX d: <https://w3id.org/icon/data/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
PREFIX sim: <https://w3id.org/simulation/ontology/>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>

SELECT DISTINCT ?artwork ?lev2Label ?CFLabel  WHERE {
VALUES ?rel {icon:hasCharacter icon:hasEvent icon:hasNamedObject icon:hasPlace icon:hasPersonification icon:hasSymbol}
?meaning ^icon:recognizedCulturalPhenomenon / ^icon:recognizedIntrinsicMeaning ?iconol .
?meaning a icon:CulturalPhenomenon;  rdfs:label ?CFLabel. 
?iconol icon:aboutWorkOfArt ?artwork .
?icrec icon:aboutWorkOfArt ?artwork; icon:recognizedImage ?img. 
?img ?rel ?lev2.
?lev2 rdfs:label ?lev2Label.  

} 
"""


result_cf2 = sparql_query_setting(q_cf2, icon_sparql)
for b in result_cf2['results']['bindings']:
    art = (b["artwork"]["value"])
    subj = (b["lev2Label"]["value"])
    cf = (b["CFLabel"]["value"])
    if cf !='?': 
        cf2_list.append([art, subj, cf])

cf2 = pd.DataFrame(np.array(cf2_list),
    columns=['Artwork', 'Level 2 subject', 'Cultural Phenomenon'])

cf2_count = cf2.groupby(["Cultural Phenomenon", "Level 2 subject"])["Artwork"].count().reset_index().sort_values(by="Artwork", ascending=False)
cf2_count.head(15)

,Cultural Phenomenon,Level 2 subject,Artwork
397,Evolution of the Iconography of Saturn,Saturn,8
663,Iconographic evolution of the representation o...,Companion of Serapis,8
624,Iconographic evolution of classical subjects,Companion of Serapis,6
925,Neoplatonic thought in visual arts,Cupid,5
398,Evolution of the Iconography of Saturn,Saturn devouring a living child,5
1229,Representation of classical content with conte...,Mercury,5
411,Evolution of the Iconography of Time,Saturn,5
621,Iconographic evolution of classical subjects,Apollo,4
575,Iconographic evolution of Cupid: in classical ...,Cupid,4
723,"In place of Serapis, his companion is represen...",Companion of Serapis,4


### Q3. 
What is the frequency with which a cultural phenomenon recognition is supported by a piece of evidence? Of which type is it? (texts, artworks, etc.)



In [24]:
evid_type_level_list = []
q_evid_type_level = '''
PREFIX data: <https://w3id.org/icon/data/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
PREFIX sim: <https://w3id.org/simulation/ontology/>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX cito: <http://purl.org/spar/cito/>

SELECT DISTINCT ?rec ?evid2 ?typeLabel  WHERE {

?rec icon:aboutWorkOfArt ?artwork; 
    cito:citesAsEvidence ?evid2; icon:recognizedIntrinsicMeaning / icon:recognizedCulturalPhenomenon ?cf.   
OPTIONAL {?evid2 a ?type. ?type rdfs:label ?typeLabel. }

}

'''


result_evid_type_level = sparql_query_setting(q_evid_type_level, icon_sparql)
for b in result_evid_type_level['results']['bindings']:
    rec =  b["rec"]["value"]
    #evid = b["evid2"]["value"]
    if "typeLabel" in b: 
        t = b["typeLabel"]["value"]
    else: 
        t = "Not given"
    evid_type_level_list.append([rec, t])

df_evid_type_level = pd.DataFrame(np.array(evid_type_level_list),
            columns=['Recognition', 'Evidence Type'])
df_evid_type_level2 = df_evid_type_level.groupby(["Evidence Type"])["Recognition"].count().sort_values(ascending=False).reset_index()  
df_evid_type_level2 

,Evidence Type,Recognition
0,Artwork,142
1,Information Object,19
2,Not given,15
3,Composition,4
4,Image,2
5,Iconological Recognition,1


### Q4. 
Is there any co-occurrence between cultural phenomena and other types of subjects?

In [26]:
artwork_lev3 = []

q_lev3 = """
PREFIX d: <https://w3id.org/icon/data/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
PREFIX sim: <https://w3id.org/simulation/ontology/>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>

SELECT DISTINCT ?artwork ?meaning ?CFBroaderLabel  WHERE {
# VALUES ?rel {icon:hasCharacter icon:hasEvent icon:hasNamedObject icon:hasPlace icon:hasPersonification icon:hasSymbol}

?meaning ^icon:recognizedCulturalPhenomenon / ^icon:recognizedIntrinsicMeaning ?iconol .
?meaning crm:P2_has_type ?CFBroader.
?iconol icon:aboutWorkOfArt ?artwork .
?CFBroader rdfs:label ?CFBroaderLabel. 


} 
""" 
result_lev3 = sparql_query_setting(q_lev3, icon_sparql)
for b in result_lev3['results']['bindings']:
    art = (b["artwork"]["value"])
    subj = (b["meaning"]["value"])
    broad = (b["CFBroaderLabel"]["value"])
    artwork_lev3.append([art, subj, broad])

cf_art_set = set() # set of the artworks having a cf of the most common categories
for l in artwork_lev3: 
    cf_art_set.add("<"+l[0]+">")


artwork_lev2 = []

q_lev2 = """
PREFIX d: <https://w3id.org/icon/data/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
PREFIX sim: <https://w3id.org/simulation/ontology/>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>

SELECT DISTINCT ?artwork ?lev2 ?typeLabel  WHERE {
VALUES ?rel {icon:hasCharacter icon:hasEvent icon:hasNamedObject icon:hasPlace icon:hasPersonification icon:hasSymbol}
?meaning ^icon:recognizedCulturalPhenomenon / ^icon:recognizedIntrinsicMeaning ?iconol .
?iconol icon:aboutWorkOfArt ?artwork .
?icrec icon:aboutWorkOfArt ?artwork; icon:recognizedImage ?img. 
?img ?rel ?lev2.
?lev2 a ?type. 
?type rdfs:label ?typeLabel. 

} 
""" 
result_lev2 = sparql_query_setting(q_lev2, icon_sparql)
for b in result_lev2['results']['bindings']:
    art = (b["artwork"]["value"])
    subj = (b["lev2"]["value"])
    broad = (b["typeLabel"]["value"])
    artwork_lev2.append([art, subj, broad])

artwork_lev1 = []

q_lev1 = """
PREFIX d: <https://w3id.org/icon/data/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
PREFIX sim: <https://w3id.org/simulation/ontology/>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>

SELECT DISTINCT ?artwork ?lev1 ?typeLabel  WHERE {
    VALUES ?rel {icon:hasFactualMeaning icon:hasExpressionalMeaning}
    ?meaning ^icon:recognizedCulturalPhenomenon / ^icon:recognizedIntrinsicMeaning ?iconol .
    ?iconol icon:aboutWorkOfArt ?artwork .
    ?artwork ^icon:aboutWorkOfArt ?icrec. 
    ?icrec a icon:PreiconographicalRecognition; icon:recognizedArtisticMotif | icon:recognizedComposition ?subj1. 
    {?subj1 ?rel ?lev1.} UNION {?subj1 icon:hasPart ?am. ?am ?rel ?lev1}
    ?lev1 a ?type. 
    ?type rdfs:label ?typeLabel. 

} 
""" 
result_lev1 = sparql_query_setting(q_lev1, icon_sparql)
for b in result_lev1['results']['bindings']:
    art = (b["artwork"]["value"])
    subj = (b["lev1"]["value"])
    broad = (b["typeLabel"]["value"])
    artwork_lev1.append([art, subj, broad])

art_described_dict = {}
for l in artwork_lev3: 
    if l[0] not in art_described_dict: 
        art_described_dict[l[0]] = set() # set: we include each tipe once per artwork
    art_described_dict[l[0]].add(l[2])
for l in artwork_lev2: 
    if l[0] in art_described_dict:  
        art_described_dict[l[0]].add(l[2])
for l in artwork_lev1: 
    if l[0] in art_described_dict: 
        art_described_dict[l[0]].add(l[2])




# we transform it in a list of lists to analyze co-occurrencies
art_described_list = [] 

for s in art_described_dict.values(): 
    l = list(s)
    art_described_list.append(l)

# co-occurrencies

dataset = art_described_list

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = apriori(df, min_support=0.2, use_colnames=True)


rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.9)
rules.antecedents = rules.antecedents.apply(lambda x: next(iter(x)))
rules.consequents = rules.consequents.apply(lambda x: next(iter(x)))

frequent_itemsets
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Action,Natural Element,0.487654,0.79321,0.487654,1.000000,1.260700,0.100842,inf
1,Character,Natural Element,0.496914,0.79321,0.478395,0.962733,1.213718,0.084238,5.548868
2,Iconographical evolution,Natural Element,0.354938,0.79321,0.342593,0.965217,1.216850,0.061052,5.945216
3,Personification,Natural Element,0.231481,0.79321,0.231481,1.000000,1.260700,0.047868,inf
4,Character,Natural Element,0.354938,0.79321,0.354938,1.000000,1.260700,0.073398,inf
5,Action,Natural Element,0.237654,0.79321,0.237654,1.000000,1.260700,0.049145,inf
6,Reception of Classical antiquity,Natural Element,0.367284,0.79321,0.367284,1.000000,1.260700,0.075951,inf
7,Character,Natural Element,0.212963,0.79321,0.209877,0.985507,1.242429,0.040952,14.268519
8,Reception of Classical antiquity,Natural Element,0.367284,0.79321,0.358025,0.974790,1.228918,0.066691,8.202675
9,Reception of Classical antiquity,Natural Element,0.274691,0.79321,0.262346,0.955056,1.204040,0.044458,4.601080


### Q5. 
What recognitions support the recognition of a cultural phenomenon?

In [32]:

    #all_levels_type_list = []
q_support = """
    PREFIX d: <https://w3id.org/icon/data/> 
    PREFIX icon: <https://w3id.org/icon/ontology/> 
    PREFIX sim: <https://w3id.org/simulation/ontology/>
    PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
    PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
    PREFIX cito: <http://purl.org/spar/cito/>

            SELECT distinct ?rec WHERE {
            ?rec icon:aboutWorkOfArt ?aboutArt; 
                cito:givesSupportTo ?citedRec.
            ?citedRec icon:recognizedIntrinsicMeaning / icon:recognizedCulturalPhenomenon ?cf2. 
        
        } 
"""
res_support = sparql_query_setting(q_support, icon_sparql) 
for b in res_support['results']['bindings']:
        rec = b["rec"]["value"]
        print(rec)



https://w3id.org/icon/data/ART1005type-MOTIFREC1
https://w3id.org/icon/data/ART1014-ICONOLREC1
https://w3id.org/icon/data/ART1016-ICONOLREC1
https://w3id.org/icon/data/ART1017-ICONOLREC3
https://w3id.org/icon/data/ART1017test-ICONOLREC1
https://w3id.org/icon/data/ART1019-MOTIFREC1
https://w3id.org/icon/data/ART1019-MOTIFREC2
https://w3id.org/icon/data/ART1019-MOTIFREC3
https://w3id.org/icon/data/ART1019-MOTIFREC4
https://w3id.org/icon/data/ART1019-MOTIFREC5
https://w3id.org/icon/data/ART1019-MOTIFREC6
https://w3id.org/icon/data/ART1036-MOTIFREC1
https://w3id.org/icon/data/ART1039-ICONOLREC2
https://w3id.org/icon/data/ART1050-ICONOLREC5
https://w3id.org/icon/data/ART1051-ICONOLREC2
https://w3id.org/icon/data/ART1051-ICONOLREC3
https://w3id.org/icon/data/ART1095-MOTIFREC1
https://w3id.org/icon/data/ART1095-MOTIFREC2
https://w3id.org/icon/data/ART1101-ICONOLREC1
https://w3id.org/icon/data/ART1102-MOTIFREC1
https://w3id.org/icon/data/ART1105-MOTIFREC1
https://w3id.org/icon/data/ART1113-MOT

### Q6. 
What are the artworks having both a cultural phenomenon in common and a different style associated?

In [34]:
q_style_cf = """
PREFIX d: <https://w3id.org/icon/data/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
PREFIX sim: <https://w3id.org/simulation/ontology/>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX cito: <http://purl.org/spar/cito/>


SELECT DISTINCT ?style ?phenomenon ?artwork ?artwork2 WHERE {
?style crm:P2_has_type <http://vocab.getty.edu/aat/300015646>. 
?artwork a icon:Artwork; 
    crm:P2_has_type ?style.
?rec icon:aboutWorkOfArt ?artwork; 
    icon:recognizedIntrinsicMeaning ?meaning. 
?meaning icon:recognizedCulturalPhenomenon ?phenomenon. 

?artwork2 a icon:Artwork; 
    crm:P2_has_type ?style.
?rec2 icon:aboutWorkOfArt ?artwork2; 
    icon:recognizedIntrinsicMeaning ?meaning2. 
?meaning2 icon:recognizedCulturalPhenomenon ?phenomenon. 
 
FILTER (?artwork != ?artwork2)
} """

res_style_cf = sparql_query_setting(q_style_cf, icon_sparql) 
for b in res_style_cf['results']['bindings']:
        style = b["style"]["value"]
        phenomenon = b["phenomenon"]["value"]
        art1 = b["artwork"]["value"]
        art2 = b["artwork2"]["value"]
        print(style, phenomenon, art1, art2)

http://vocab.getty.edu/aat/300020770 https://w3id.org/icon/data/CF1240 https://w3id.org/icon/data/ART1088 https://w3id.org/icon/data/ART1093
http://vocab.getty.edu/aat/300020770 https://w3id.org/icon/data/CF1240 https://w3id.org/icon/data/ART1089 https://w3id.org/icon/data/ART1093
http://vocab.getty.edu/aat/300020770 https://w3id.org/icon/data/CF1240 https://w3id.org/icon/data/ART1088 https://w3id.org/icon/data/ART1095
http://vocab.getty.edu/aat/300020770 https://w3id.org/icon/data/CF1240 https://w3id.org/icon/data/ART1089 https://w3id.org/icon/data/ART1095
http://vocab.getty.edu/aat/300020770 https://w3id.org/icon/data/CF1240 https://w3id.org/icon/data/ART1093 https://w3id.org/icon/data/ART1095
http://vocab.getty.edu/aat/300020770 https://w3id.org/icon/data/CF1214 https://w3id.org/icon/data/ART1089 https://w3id.org/icon/data/ART1088
http://vocab.getty.edu/aat/300020770 https://w3id.org/icon/data/CF1214 https://w3id.org/icon/data/ART1088 https://w3id.org/icon/data/ART1089
http://vocab.

## Symbols

### Q7. 
Which and how many symbols express the same concept?

In [43]:
symb_list = []
q_symb = """
PREFIX d: <https://w3id.org/icon/data/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
PREFIX sim: <https://w3id.org/simulation/ontology/>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX cito: <http://purl.org/spar/cito/>

SELECT DISTINCT ?simulacrumL ?realityL WHERE {
VALUES ?rel {icon:hasCharacter icon:hasEvent icon:hasNamedObject icon:hasPlace icon:hasPersonification crm:P138_represents}


?img icon:hasSymbol ?simulation.
?simulation sim:hasSimulacrum ?simulacrum; sim:hasRealityCounterpart ?reality. 
?simulacrum rdfs:label ?simulacrumL.
?reality rdfs:label ?realityL. 
}
"""

result_symb = sparql_query_setting(q_symb, icon_sparql)
for b in result_symb['results']['bindings']:
    sim = (b["simulacrumL"]["value"])
    rel = (b["realityL"]["value"])
    

    symb_list.append([sim, rel])

        # create a dataframe
df_symb = pd.DataFrame(np.array(symb_list),
    columns=['Symbol', 'Symbolic Meaning'])

df_symb_meaning = df_symb.groupby(['Symbolic Meaning'])['Symbol'].count().sort_values(ascending=False).reset_index()

print("How many symbols express a concept")
df_symb_meaning.head(15)

How many symbols express a concept


,Symbolic Meaning,Symbol
0,the future,3
1,the past,3
2,the present,3
3,"three forms of time (past, present, future)",3
4,attribute of wisdom,3
5,love,3
6,avarice,2
7,pride,2
8,persuasion,2
9,fertility,2


In [44]:
print("Which symbols express a concept")
df_symb2 = df_symb
df_symb2['Symbol']= df_symb2.groupby(['Symbolic Meaning'])['Symbol'].transform(lambda x: ', '.join(x))
df_symb2.drop_duplicates()
df_symb2.head(20)

Which symbols express a concept


,Symbol,Symbolic Meaning
0,"cone shell, puer mingens",fertility
1,closed cash box,Saturninian parsimony
2,children crouching down,unborn souls doomed to descend to the lower sp...
3,hares,"animal love, fertility"
4,fettered ape,blind and unhappy human soul
5,fly,decade
6,Ganymede ascending to Heaven on the wings of a...,ecstasy of Platonic love
7,fire,fire to burn the time
8,four wings,four seasons
9,glass sphere,God


### Q8. 
What different symbolical meanings can the same symbol have?

In [45]:
df_symb1 = df_symb.groupby(['Symbol'])['Symbolic Meaning'].count().sort_values(ascending=False).reset_index()
print("How many symbolical meanings correspond to the same symbol?")
df_symb1.head(15)      

How many symbolical meanings correspond to the same symbol?


,Symbol,Symbolic Meaning
0,"putti with dolphins, knotted ribbon bracelets,...",3
1,"head of a wolf, face of an old man, face of an...",3
2,"head of a lion, face of an adult man, face of ...",3
3,"head of a dog, face of a young man, face of a ...",3
4,glass sphere,3
5,four Times of Day,3
6,"owl, book, serpent",3
7,"three headed companion of Serapis, three books...",3
8,"cone shell, puer mingens",2
9,"ape, Venus, Cupid, Vulcan",2


In [47]:
print("Which symbolic meanings correspond to a symbol")
df_symb3 = df_symb
df_symb3['Symbolic Meaning']= df_symb3.groupby(['Symbol'])['Symbolic Meaning'].transform(lambda x: ', '.join(x))
df_symb3.drop_duplicates()
df_symb3.head(20)

Which symbolic meanings correspond to a symbol


,Symbol,Symbolic Meaning
0,"cone shell, puer mingens","fertility, fertility, fertility, fertility"
1,closed cash box,Saturninian parsimony
2,children crouching down,unborn souls doomed to descend to the lower sp...
3,hares,"animal love, fertility"
4,fettered ape,"blind and unhappy human soul, subjugation of b..."
5,fly,decade
6,Ganymede ascending to Heaven on the wings of a...,ecstasy of Platonic love
7,fire,"fire to burn the time, luxury, fire to burn th..."
8,four wings,four seasons
9,glass sphere,"God, harmony that can be easily broken, Heaven..."


### Q9. 
How do symbols evolve over time and in different contexts?

In [50]:
symb_time_list = []

q_l1_l2_time = """
PREFIX d: <https://w3id.org/icon/data/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
PREFIX sim: <https://w3id.org/simulation/ontology/>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX cito: <http://purl.org/spar/cito/>

SELECT DISTINCT ?simulacrumL ?realityL ?contextL ?startDate ?endDate WHERE {
?rec icon:aboutWorkOfArt ?artwork; icon:recognizedImage / icon:hasSymbol ?simulation.
?simulation sim:hasSimulacrum ?simulacrum; sim:hasRealityCounterpart ?reality. 
?simulacrum rdfs:label ?simulacrumL.
?reality rdfs:label ?realityL. 


?obj crm:P65_shows_visual_item ?artwork; 
    ^crm:P108_has_produced / crm:P4_has_time_span ?time . 
?time crm:P82a_begin_of_the_begin ?startDate; 
    crm:P82b_end_of_the_end ?endDate. 

OPTIONAL {?simulation sim:hasSimulationContext ?context. ?context rdfs:label ?contextL}
}
"""


res_l1_l2_time = sparql_query_setting(q_l1_l2_time, icon_sparql) 
for b in res_l1_l2_time['results']['bindings']:
    simulacrum = b["simulacrumL"]["value"]
    reality = b["realityL"]["value"]
    start = b["startDate"]["value"]
    end = b["endDate"]["value"]
    if "contextL" in b:
        context = b["contextL"]["value"]
    else: 
        context = ""

    symb_time_list.append([simulacrum, reality,context, start, end])

df_symb_time = pd.DataFrame(np.array(symb_time_list),
columns=['Symbol','Symbolic meaning', 'Context', 'Start Date', 'End Date'])

df_symb_time.head(15)
        


,Symbol,Symbolic meaning,Context,Start Date,End Date
0,bull,Christ,Medieval Mithography,1351-01-01T0:00:00,1375-12-31T23:59:59
1,Europa,human soul,Medieval Mithography,1351-01-01T0:00:00,1375-12-31T23:59:59
2,face of a young woman,the future,Middle Ages,1368-01-01T0:00:00,1400-12-31T23:59:59
3,face of an adult woman,the present,Middle Ages,1368-01-01T0:00:00,1400-12-31T23:59:59
4,face of an old woman,the past,Middle Ages,1368-01-01T0:00:00,1400-12-31T23:59:59
5,red cloth,bridal chamber decoration,,1520-01-01T0:00:00,1520-12-31T23:59:59
6,playful attitude,"joy, love",,1520-01-01T0:00:00,1520-12-31T23:59:59
7,puer mingens,fertility,,1520-01-01T0:00:00,1520-12-31T23:59:59
8,ox,Christ,,1532-01-01T0:00:00,1532-12-31T23:59:59
9,ox carrying off his master,Luke the Evangelist obeys the dictates of the ...,,1532-01-01T0:00:00,1532-12-31T23:59:59


### Q10. 
What are all the symbols motivated by a specific source? Do they vary from the corresponding subjects not citing a piece of evidence?

In [52]:
symb_evid_list = []

q_symb_evid = """
PREFIX d: <https://w3id.org/icon/data/> 
PREFIX icon: <https://w3id.org/icon/ontology/> 
PREFIX sim: <https://w3id.org/simulation/ontology/>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX cito: <http://purl.org/spar/cito/>

SELECT DISTINCT ?simulacrumL ?realityL ?evid  WHERE {
?rec icon:aboutWorkOfArt ?artwork; icon:recognizedImage / icon:hasSymbol ?simulation.
?simulation sim:hasSimulacrum ?simulacrum; sim:hasRealityCounterpart ?reality. 
?simulacrum rdfs:label ?simulacrumL.
?reality rdfs:label ?realityL. 
 

OPTIONAL {?simulation cito:citesAsEvidence ?evid.}
OPTIONAL {?rec cito:citesAsEvidence ?evid.}
}
"""


res_symb_evid = sparql_query_setting(q_symb_evid, icon_sparql) 
for b in res_symb_evid['results']['bindings']:
    simulacrum = b["simulacrumL"]["value"]
    reality = b["realityL"]["value"]
    if "evid" in b:
        evid = "Yes"
    else: 
        evid = "No"

    symb_evid_list.append([simulacrum, reality,evid])

df_symb_evid = pd.DataFrame(np.array(symb_evid_list),
columns=['Symbol','Symbolic meaning', 'Evidence']).sort_values(by='Symbol')

df_symb_evid.head(15)
        


,Symbol,Symbolic meaning,Evidence
147,Angel smiling while carrying the coffin of the...,angel glad that the soul of the Pope had been ...,Yes
148,Angel wepting while carrying the coffin of the...,angel grieved that the world had been deprived...,Yes
182,Apollo,Idolatry,No
185,Bacchus,Folly,No
141,Blind Fortune blindfolding Cupid and putting h...,affection changes with luck,No
121,Cameo with a pair of winged horses and a chari...,Plato's immortal comparison of the human soul,No
117,Cupid,amor carnalis,No
143,Cupid between Venus Coelestis and Venus Vulgaris,love acts as an intermediary between heaven an...,Yes
122,Cupid unblindfolding himself,Cupid transforming himself from a personificat...,No
165,"Draped Ganymede raped by the eagle, so that he...",perfect harmony of soul and body,No


### Q11. 
Do deeper meanings cite as support textual sources?

In [53]:

    #all_levels_type_list = []
q_concept_evid = """
    PREFIX d: <https://w3id.org/icon/data/> 
    PREFIX icon: <https://w3id.org/icon/ontology/> 
    PREFIX sim: <https://w3id.org/simulation/ontology/>
    PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
    PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
    PREFIX cito: <http://purl.org/spar/cito/>

            SELECT distinct ?rec WHERE {
            ?rec icon:aboutWorkOfArt ?aboutArt; 
                icon:recognizedIntrinsicMeaning / icon:recognizedConceptualObject ?concept; 
                cito:citesAsEvidence ?evid.
                ?evid a crm:E73_Information_Object. 

        
        } 
"""
res_concept_evid = sparql_query_setting(q_concept_evid, icon_sparql) 
for b in res_concept_evid['results']['bindings']:
        rec = b["rec"]["value"]
        print(rec)



https://w3id.org/icon/data/ART1011test-ICONOLREC1
https://w3id.org/icon/data/ART1011test-ICONOLREC2
https://w3id.org/icon/data/ART1012test-ICONOLREC1
https://w3id.org/icon/data/ART1012test-ICONOLREC2
https://w3id.org/icon/data/ART1386-ICONOLREC1


## Iconographies

### Q12. 
Which are the representational variations of iconography? I.e. what are the sets of level 1 subjects composing the recognized level 2 subject in artworks?

### Q13. 
What are the attributes having a symbolic meaning?

### Q14. 
How does the representation of iconography evolve over time? Of which level 1 subjects is it composed?



### Q15. 
Which attributes allow us to identify representations? What is their frequency?

### Q16. 
Among the attributes marked as recognising, which are the most common and rare?

### Q17. 
Which are the most common level 1 subjects not marked as recognising?

## Evidence

### Q18. 
What were the known textual sources to which the artwork refers? Is the artwork involved in a cultural phenomenon?

## Visual Citation

### Q19. 
What artworks cite the visual pattern of other ones?

### Q20. 
In which cases artworks involved in a visual citation have a cultural phenomenon associated?